# Map Reduce Lab

### Introduction

In this lesson, we'll practice working with Pyspark by looking at sales at different grocery store chains.  Let's get started.

### Getting Setup (On Google Colab)

* Begin by installing some pip packages and the java development kit.

In [ ]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet 
!apt install openjdk-8-jdk-headless &> /dev/null

* Then set the java environmental variable

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

* Then connect to a SparkSession, setting the spark ui port to `4050`.

In [ ]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050').setAppName("netflix").setMaster("local[2]")
sc = SparkContext.getOrCreate(conf=conf)

* Then we need to install ngrok which will allow us to place our local spark ui on the web.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip &> /dev/null
!unzip ngrok-stable-linux-amd64.zip &> /dev/null
get_ipython().system_raw('./ngrok http 4050 &')

* And finally we get a link our Spark UI

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Creating our RDD

Now to create our RDD, we'll load in some data from a pandas dataframe, and convert this dataframe to a list of dictionaries.

In [ ]:
import pandas as pd

df = pd.read_csv('s3://jigsaw-labs-student/supermarket_sales.csv')
sales = df.to_dict('records')

In [ ]:
sales[:1]

[{'Invoice ID': '750-67-8428',
  'Branch': 'A',
  'City': 'Yangon',
  'Customer type': 'Member',
  'Gender': 'Female',
  'Product line': 'Health and beauty',
  'Unit price': 74.69,
  'Quantity': 7,
  'Tax 5%': 26.1415,
  'Total': 548.9715,
  'Date': '1/5/2019',
  'Time': '13:08',
  'Payment': 'Ewallet',
  'cogs': 522.83,
  'gross margin percentage': 4.761904762,
  'gross income': 26.1415,
  'Rating': 9.1}]

Ok, so currently, our `sales` is a list of dictionaries.  Turn this list of dictionaries into an RDD by calling the `parallelize` method and passing in the list of dictionaries.

In [ ]:
sales_rdd = None

In [ ]:
sales_rdd
# ParallelCollectionRDD[4] at readRDDFromFile at PythonRDD

ParallelCollectionRDD[11] at readRDDFromFile at PythonRDD.scala:274

> We can see that we have 1000 records.

In [ ]:
sales_rdd.count()

1000

Now let's see the number of partitions for our rdd.

In [ ]:

# 2

2

Ok, so now let's filter for only branch b (below you only need to use the `filter` method.  We'll call the `collect` method for you later on).

In [ ]:
branch_b = None

In [ ]:
branch_b.collect()[0]
# [{'Invoice ID': '692-92-5582',
#   'Branch': 'B',
#   'City': 'Mandalay',
#   'Customer type': 'Member',
#   'Gender': 'Female',
#   'Product line': 'Food and beverages',
#   'Unit price': 54.84,
#   'Quantity': 3,
#   'Tax 5%': 8.226,
#   'Total': 172.746,
#   'Date': '2/20/2019',
#   'Time': '13:27',
#   'Payment': 'Credit card',
#   'cogs': 164.52,
#   'gross margin percentage': 4.761904762,
#   'gross income': 8.226,
#   'Rating': 5.9}]

{'Invoice ID': '692-92-5582',
 'Branch': 'B',
 'City': 'Mandalay',
 'Customer type': 'Member',
 'Gender': 'Female',
 'Product line': 'Food and beverages',
 'Unit price': 54.84,
 'Quantity': 3,
 'Tax 5%': 8.226,
 'Total': 172.746,
 'Date': '2/20/2019',
 'Time': '13:27',
 'Payment': 'Credit card',
 'cogs': 164.52,
 'gross margin percentage': 4.761904762,
 'gross income': 8.226,
 'Rating': 5.9}

Ok, so from here, let's take a look at the spark ui.  We can look at the spark ui by simply typing in the spark context, and from there clicking on the link to the spark ui. 

> We can also directly go to http://localhost:4040, which is the designated port for spark ui.

In [ ]:
sc

<SparkContext master=local[2] appName=groceryStores>

When we click on the spark ui, we should see a number of jobs listed.  As we know, the job  at the top is the most recent job that was called.  Click on the most recent job and view the DAG.

<img src="https://github.com/jigsawlabs-student/pyspark-map-reduce-lab/blob/main/dag_parallel.png?raw=1" width="20%">

From there, click on the stage that has parallelize to dig a bit deeper into this.

<img src="https://github.com/jigsawlabs-student/pyspark-map-reduce-lab/blob/main/dag_collect.png?raw=1" width="40%">

So here it looks like Spark has first created a partitioned dataset with the RDD.  It says `readRDDFromFile`, but this is when it reads from our list of dictionaries. And then we can see that we returned a PythonRDD.  This is the list of dictionaries that was returned to us when we called `collect()`. 

> So above, Spark applied the filter function across the two partitions of the dataset, and then returned the results in the Python list of dictionaries in Python.  

We can see that this was performed in parallel, if we scroll down and view the event timeline.

<img src="https://github.com/jigsawlabs-student/pyspark-map-reduce-lab/blob/main/green_parallel.png?raw=1" width="80%">

So notice, here, we see two lines, each of them represents a task run on a separate core.  Also, notice that these tasks overlab with one another, indicating that they were run in parallel.

> If we look at the color coding, we can see that we see a blue for scheduling of tasks, and a longer green line for the compute time.

### Seeing our Shuffle

Ok, now let's run a procedure that will result in shuffle.  For us, this is our group by.  We can begin with our `sales_rdd`, and from there let's group by the stores, and count the number of sales in each store.

> To see the results, end with a call to `collect()`.

In [ ]:


# [('C', 328), ('A', 340), ('B', 332)]

[('C', 328), ('A', 340), ('B', 332)]

After getting the correct result, take a look at the spark ui to see what occurred in the job.  For our most recent job, we should see something like the following:

<img src="https://github.com/jigsawlabs-student/pyspark-map-reduce-lab/blob/main/dag_viz.png?raw=1" width="40%">

And then below we see the following.

<img src="https://github.com/jigsawlabs-student/pyspark-map-reduce-lab/blob/main/completed_stage.png?raw=1" width="90%">

So the first step was the `groupBy`, which occurred in parallel.  And then in the following stage partitionBy and mapPartitions occurred --  this was the count function. Let's take a closer look at the first stage -- which has the `parallelize`.

<img src="https://github.com/jigsawlabs-student/pyspark-map-reduce-lab/blob/main/groupby_stage.png?raw=1" width="60%">

And on the same panel we can see the event timeline.

<img src="https://github.com/jigsawlabs-student/pyspark-map-reduce-lab/blob/main/shuffle_event.png?raw=1" width="100%">

So looking at the event timeliene, we can see that first the compute finds the records by the branch, and then, in yellow, we see that the shuffle occurs to repartition the the records by key.  

### Summary

In this lab, we practiced working with Pyspark, and using the Spark UI to see the operations that occur, even when we call relatively simple methods.  We saw that by partitioning our data, methods like `filter` are performed in parallel.  And we can see that by looking at the tasks performede in a stage, and seeing that they occur simultaneously.  

Then, we performed a groupby, which as we know results in shuffling.  This means that our data needs to be repartitioned, and often transferred between worker nodes to perform the operation.  We saw the shuffling occur, by taking a closer look at the stage that performed the groupby and looking at the event timeline.

<img src="https://github.com/jigsawlabs-student/pyspark-map-reduce-lab/blob/main/shuffle_event.png?raw=1" width="100%">

### Resouorces

[From Google Cloud to Pyspark](https://medium.com/@kashif.sohail/read-files-from-google-cloud-storage-bucket-using-local-pyspark-and-jupyter-notebooks-f8bd43f4b42e)